# Similarity Algorithms - Usage Examples

This notebook demonstrates all similarity algorithms available in the `granular_impact.similarity` module.

## Overview

The similarity module provides multiple algorithms for computing text similarity:

- **Jaccard**: Token set overlap similarity
- **Difflib**: Sequence matching (Ratcliff/Obershelp)
- **BM25**: Probabilistic ranking function
- **Hybrid**: Weighted combination of multiple algorithms

All algorithms are optimized for FAQ content analysis and change detection.

## Setup and Imports

## Prerequisites

Before running this notebook, ensure that:

1. **The FAQ project is installed**:
   ```bash
   # From the project root (FAQ_update/)
   pip install -e .
   ```

2. **Required dependencies are available**:
   - Python 3.8+
   - All dependencies from `pyproject.toml` or `requirements.txt`

3. **You're running from the correct directory**:
   - Either from project root or from `granular_impact/similarity/`
   - The notebook will automatically adjust the Python path

In [1]:
# Add parent directory to path for imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import all similarity calculators
from granular_impact.similarity import (
    # Base classes
    BaseSimilarityCalculator,
    SimilarityResult,
    
    # Algorithms
    JaccardSimilarityCalculator,
    WeightedJaccardSimilarityCalculator,
    DifflibSimilarityCalculator,
    TokenBasedDifflibCalculator,
    BM25SimilarityCalculator,
    HybridSimilarityCalculator,
    PolicyContentSimilarity,
)

# Import factory functions
from granular_impact.similarity.difflib_sim import (
    create_faq_modification_detector,
    create_character_level_analyzer,
    create_fast_diff_calculator,
)

import json
from pprint import pprint

print("✓ All modules imported successfully!")

✓ All modules imported successfully!


## Sample FAQ Content

We'll use realistic FAQ markdown content for our examples.

In [2]:
# Original FAQ content
faq_original = """
# Employee Vacation Policy

## How many vacation days do employees receive?

Full-time employees receive **15 vacation days** per year. Vacation days accrue monthly 
and can be used after 90 days of employment. Unused vacation days can be carried over 
to the next year, up to a maximum of 5 days.

### Eligibility

- Must be a full-time employee
- Must have completed 90-day probationary period
- Part-time employees receive prorated vacation days
"""

# Minor modification (number change)
faq_minor_change = """
# Employee Vacation Policy

## How many vacation days do employees receive?

Full-time employees receive **20 vacation days** per year. Vacation days accrue monthly 
and can be used after 90 days of employment. Unused vacation days can be carried over 
to the next year, up to a maximum of 5 days.

### Eligibility

- Must be a full-time employee
- Must have completed 90-day probationary period
- Part-time employees receive prorated vacation days
"""

# Major rewrite (different topic)
faq_major_change = """
# Vehicle Parking Policy

## Where can employees park their vehicles?

Employees can park in designated parking areas on floors 2-5 of the parking structure. 
A valid parking permit must be displayed at all times. Monthly parking passes are 
available for $50 per month.

### Parking Rules

- Display parking permit visibly
- Park only in designated employee spaces
- Visitor parking is available on the first floor
"""

# Identical content
faq_identical = faq_original

print("✓ Sample FAQ content loaded")

✓ Sample FAQ content loaded


---

# 1. Jaccard Similarity

**Jaccard similarity** measures the overlap between two token sets:

```
J(A, B) = |A ∩ B| / |A ∪ B|
```

- **Best for**: Detecting word-level changes, keyword matching
- **Strengths**: Fast, simple, good for FAQ content with specific terms
- **Limitations**: Doesn't consider word order or semantic meaning

## 1.1 Basic Jaccard Similarity

In [3]:
# Create Jaccard calculator
jaccard_calc = JaccardSimilarityCalculator(
    ngram_size=1,  # Unigrams (individual words)
    lowercase=True,
    remove_punctuation=False,  # Keep punctuation for FAQ content
)

# Compare identical texts
result = jaccard_calc.compute_similarity(faq_original, faq_identical)
print(f"Identical FAQs: {result.score:.4f}")
print(f"Algorithm: {result.algorithm}")
print(f"\nMetadata:")
pprint(result.metadata)

Identical FAQs: 1.0000
Algorithm: jaccard

Metadata:
{'intersection_size': 50,
 'ngram_size': 1,
 'set1_size': 50,
 'set2_size': 50,
 'union_size': 50,
 'use_character_ngrams': False}


In [4]:
# Compare minor change (15 -> 20 vacation days)
result = jaccard_calc.compute_similarity(faq_original, faq_minor_change)
print(f"Minor change (15→20 days): {result.score:.4f}")
print(f"Intersection size: {result.metadata['intersection_size']}")
print(f"Union size: {result.metadata['union_size']}")
print(f"Set difference: {result.metadata['union_size'] - result.metadata['intersection_size']} tokens")

Minor change (15→20 days): 0.9608
Intersection size: 49
Union size: 51
Set difference: 2 tokens


In [5]:
# Compare major change (vacation -> parking)
result = jaccard_calc.compute_similarity(faq_original, faq_major_change)
print(f"Major change (different topic): {result.score:.4f}")
print(f"\nThis is correctly identified as different content!")

Major change (different topic): 0.1786

This is correctly identified as different content!


## 1.2 Jaccard with N-grams

In [6]:
# Bigram Jaccard (considers word pairs)
jaccard_bigram = JaccardSimilarityCalculator(
    ngram_size=2,  # Bigrams
    lowercase=True,
    remove_punctuation=False,
)

result_unigram = jaccard_calc.compute_similarity(faq_original, faq_minor_change)
result_bigram = jaccard_bigram.compute_similarity(faq_original, faq_minor_change)

print(f"Unigram Jaccard: {result_unigram.score:.4f}")
print(f"Bigram Jaccard:  {result_bigram.score:.4f}")
print(f"\nBigrams are more sensitive to word order and phrase changes.")

Unigram Jaccard: 0.9608
Bigram Jaccard:  0.9412

Bigrams are more sensitive to word order and phrase changes.


## 1.3 FAQ-Optimized Jaccard (Factory Method)

In [7]:
# Use factory method for FAQ content
faq_jaccard = JaccardSimilarityCalculator.for_faq_content()

# This preserves numbers and dates
text1 = "Employee deadline is January 15, 2024"
text2 = "Employee deadline is February 15, 2024"

result = faq_jaccard.compute_similarity(text1, text2)
print(f"FAQ-optimized Jaccard: {result.score:.4f}")
print(f"\nPreserves numbers and dates for accurate FAQ comparison.")

FAQ-optimized Jaccard: 0.7143

Preserves numbers and dates for accurate FAQ comparison.


## 1.4 Weighted Jaccard Similarity

In [8]:
# Create weighted Jaccard with emphasis on important terms
weighted_jaccard = WeightedJaccardSimilarityCalculator(
    token_weights={
        "vacation": 3.0,  # Higher weight for key term
        "days": 2.0,
        "employees": 2.0,
        "15": 2.5,  # Numbers are important
        "20": 2.5,
    },
    default_weight=1.0,
)

result_standard = jaccard_calc.compute_similarity(faq_original, faq_minor_change)
result_weighted = weighted_jaccard.compute_similarity(faq_original, faq_minor_change)

print(f"Standard Jaccard:  {result_standard.score:.4f}")
print(f"Weighted Jaccard:  {result_weighted.score:.4f}")
print(f"\nWeighted version emphasizes important domain-specific terms.")

Standard Jaccard:  0.9608
Weighted Jaccard:  0.9000

Weighted version emphasizes important domain-specific terms.


---

# 2. Difflib Similarity

**Difflib** uses the Ratcliff/Obershelp algorithm for sequence matching:

- **Best for**: Character-level changes, typo detection, precise text comparison
- **Strengths**: Detects small character changes ("10" → "12"), provides detailed diff
- **Limitations**: Slower for large texts, character-focused

## 2.1 Character-Level Difflib

In [9]:
# Create difflib calculator
difflib_calc = DifflibSimilarityCalculator(
    autojunk=True,
    lowercase=True,
    remove_punctuation=False,
)

# Compare texts
result = difflib_calc.compute_similarity(faq_original, faq_minor_change)
print(f"Difflib similarity: {result.score:.4f}")
print(f"\nMetadata:")
print(f"  Ratio: {result.metadata['ratio']:.4f}")
print(f"  Matching blocks: {result.metadata['matching_blocks']}")
print(f"  Matching characters: {result.metadata['matching_characters']}")
print(f"\nOperations:")
pprint(result.metadata['operations'])

Difflib similarity: 0.9955

Metadata:
  Ratio: 0.9955
  Matching blocks: 3
  Matching characters: 440

Operations:
{'delete': 0, 'equal': 2, 'insert': 0, 'replace': 1, 'total_operations': 3}


## 2.2 Diff Blocks (Detailed Changes)

In [10]:
# Get detailed diff blocks
text1 = "Employees receive 15 vacation days per year"
text2 = "Employees receive 20 vacation days per year"

diff_blocks = difflib_calc.get_diff_blocks(text1, text2)

print("Diff Blocks:")
print("=" * 60)
for block in diff_blocks:
    if block.tag == 'replace':
        print(f"REPLACE: '{block.text1}' → '{block.text2}'")
    elif block.tag == 'equal':
        if len(block.text1) < 30:
            print(f"EQUAL:   '{block.text1}'")
    elif block.tag == 'delete':
        print(f"DELETE:  '{block.text1}'")
    elif block.tag == 'insert':
        print(f"INSERT:  '{block.text2}'")

Diff Blocks:
EQUAL:   'employees receive '
REPLACE: '15' → '20'
EQUAL:   ' vacation days per year'


## 2.3 Unified Diff (Line-by-Line)

In [11]:
# Generate unified diff
unified_diff = difflib_calc.get_unified_diff(
    faq_original,
    faq_minor_change,
    context_lines=2
)

print("Unified Diff:")
print("=" * 60)
for line in unified_diff[:15]:  # Show first 15 lines
    print(line)

Unified Diff:
--- text1
+++ text2
@@ -4,5 +4,5 @@
 ## How many vacation days do employees receive?

 

-Full-time employees receive **15 vacation days** per year. Vacation days accrue monthly 

+Full-time employees receive **20 vacation days** per year. Vacation days accrue monthly 

 and can be used after 90 days of employment. Unused vacation days can be carried over 

 to the next year, up to a maximum of 5 days.



## 2.3 Unified Diff (Line-by-Line) - OLD FORMAT

**Note**: For LLM prompts, use the new `get_llm_friendly_diff()` method shown in section 2.3.1 above instead!

In [12]:
# Example 4: Using in an LLM prompt
text_old = "The maximum sick leave is 10 days per year. Contact HR for extensions."
text_new = "The maximum sick leave is 15 days per year. Contact HR for extensions."

diff_for_llm = difflib_calc.get_llm_friendly_diff(text_old, text_new)

# This is how you'd use it in an LLM prompt
llm_prompt = f"""
Please review the following policy change and summarize the impact:

{diff_for_llm}

Provide a brief summary of what changed and who it affects.
"""

print("Example LLM Prompt:")
print("=" * 70)
print(llm_prompt)
print("=" * 70)
print("\nThis format makes it easy for LLMs to understand exactly what changed!")

Example LLM Prompt:

Please review the following policy change and summarize the impact:

Found 1 change(s):

CHANGE #1:
Context before: "the maximum sick leave is 1"
BEFORE: "0"
AFTER:  "5"
Context after: "days per year. contact hr for extensions."

Provide a brief summary of what changed and who it affects.


This format makes it easy for LLMs to understand exactly what changed!


In [13]:
# Example 3: Simple inline change
text1 = "Employees receive 15 vacation days per year"
text2 = "Employees receive 20 vacation days per year"

simple_diff = difflib_calc.get_llm_friendly_diff(text1, text2)

print("Simple Change Example:")
print("=" * 70)
print(simple_diff)
print("\n" + "=" * 70)

# Compare with old unified diff format
print("\nOLD FORMAT (unified diff) - harder to read:")
print("-" * 70)
unified = difflib_calc.get_unified_diff(text1, text2, context_lines=0)
for line in unified:
    print(line)

Simple Change Example:
Found 1 change(s):

CHANGE #1:
Context before: "employees receive"
BEFORE: "15"
AFTER:  "20"
Context after: "vacation days per year"


OLD FORMAT (unified diff) - harder to read:
----------------------------------------------------------------------
--- text1
+++ text2
@@ -1 +1 @@
-Employees receive 15 vacation days per year
+Employees receive 20 vacation days per year


In [ ]:
# RECOMMENDED: Line-based diff with inline highlighting
# This shows full sentences + what changed!

faq_with_two_changes = """
# Employee Vacation Policy

## How many vacation days do employees receive?

Full-time employees receive **20 vacation days** per year. Vacation days accrue monthly
and can be used after 90 days of employment. Unused vacation days can be carried over
to the next year, up to a maximum of 10 days.

### Eligibility

- Must be a full-time employee
- Must have completed 90-day probationary period
- Part-time employees receive prorated vacation days
"""

llm_diff_improved = difflib_calc.get_llm_friendly_diff_line_based(
    faq_original,
    faq_with_two_changes,
    context_lines=1,
    show_inline_changes=True  # Shows "WHAT CHANGED" section
)

print("⭐ RECOMMENDED: Line-Based Diff with Inline Highlighting")
print("=" * 70)
print(llm_diff_improved)
print("=" * 70)
print("\n✅ PERFECT FOR LLM PROMPTS!")
print("- Shows full sentence in BEFORE and AFTER")
print("- Separate 'WHAT CHANGED' section highlights exact differences") 
print("- Clean, easy to understand format")

In [15]:
# Example 1: Character-level diff (good for inline changes)
llm_diff = difflib_calc.get_llm_friendly_diff(
    faq_original,
    faq_minor_change,
    context_lines=2,
    max_context_chars=80
)

print("LLM-Friendly Diff (Character-level):")
print("=" * 70)
print(llm_diff)
print("\n" + "=" * 70)
print("\nThis format is ideal for LLM prompts - clear, concise, and focused!")

LLM-Friendly Diff (Character-level):
Found 1 change(s):

CHANGE #1:
Context before: "...y ## how many vacation days do employees receive? full-time employees receive **"
BEFORE: "15"
AFTER:  "20"
Context after: "vacation days** per year. vacation days accrue monthly and can be used after 90..."


This format is ideal for LLM prompts - clear, concise, and focused!


---

## 2.3.2 Complete LLM-Friendly Diff Examples

This section shows comprehensive examples of using the LLM-friendly diff formats for passing changes to language models.

**Note:** If you get a `TypeError` about unexpected keyword arguments, run the cell below first to reload the module with the latest updates.

In [ ]:
# Reload the module to get the latest changes
import importlib
import granular_impact.similarity.difflib_sim
importlib.reload(granular_impact.similarity.difflib_sim)

# Re-import the classes
from granular_impact.similarity.difflib_sim import DifflibSimilarityCalculator

# Recreate the calculator with the updated code
difflib_calc = DifflibSimilarityCalculator(
    autojunk=True,
    lowercase=True,
    remove_punctuation=False,
)

print("✓ Module reloaded with latest LLM-friendly diff methods!")

In [ ]:
### Summary: When to Use Each Method

print("\n" + "=" * 80)
print("SUMMARY: When to Use Each Method")
print("=" * 80)

summary = """
1. get_llm_friendly_diff() - Character-level
   - Best for: Short text, inline changes, single-line comparisons
   - Shows: Context snippets with BEFORE/AFTER
   - Example: Detecting number changes in a sentence

2. get_llm_friendly_diff_line_based() - Line-based with inline highlighting ⭐ RECOMMENDED
   - Best for: Multi-line documents, FAQs, markdown, structured content
   - Shows: Full sentences in BEFORE/AFTER + "WHAT CHANGED" section
   - Example: Comparing FAQ documents, policy updates

KEY FEATURES:
✅ Full sentence (stops at period) in BEFORE and AFTER
✅ Separate "WHAT CHANGED" section showing exact differences
✅ Relevant context lines
✅ Clean format perfect for LLM prompts
✅ No confusing diff symbols (+, -, @@)

USAGE TIP:
Always use show_inline_changes=True (default) to get the "WHAT CHANGED" section.
This makes it crystal clear to the LLM what exactly changed!
"""

print(summary)

In [ ]:
### Example 3: Best Format for LLM Prompts ⭐ RECOMMENDED

print("\n" + "=" * 80)
print("EXAMPLE 3: Recommended Format for LLM Prompts")
print("=" * 80)
print("This is the BEST format to use when passing diffs to LLMs:\n")

# Use line-based with inline changes
line_diff = difflib_calc.get_llm_friendly_diff_line_based(
    faq_original,
    faq_minor_change,
    context_lines=1,
    show_inline_changes=True
)

# Create a complete LLM prompt
llm_prompt = f"""You are analyzing policy document changes.

CHANGES DETECTED:
{line_diff}

TASKS:
1. Summarize what changed in plain English
2. Identify who is affected by these changes
3. Assess the impact (positive/negative/neutral)
4. Suggest if employees should be notified

Provide your analysis in a structured format."""

print(llm_prompt)

In [ ]:
### Example 2: Multiple Changes in FAQ Document

print("\n" + "=" * 80)
print("EXAMPLE 2: Multiple Changes in FAQ Document")
print("=" * 80)
print("Scenario: FAQ policy updated with new vacation day limits\n")

faq_with_multiple_changes = """
# Employee Vacation Policy

## How many vacation days do employees receive?

Full-time employees receive **20 vacation days** per year. Vacation days accrue monthly
and can be used after 90 days of employment. Unused vacation days can be carried over
to the next year, up to a maximum of 10 days.

### Eligibility

- Must be a full-time employee
- Must have completed 90-day probationary period
- Part-time employees receive prorated vacation days
"""

diff = difflib_calc.get_llm_friendly_diff_line_based(
    faq_original,
    faq_with_multiple_changes,
    context_lines=1,
    show_inline_changes=True
)
print(diff)

In [ ]:
### Example 1: Simple Inline Change

print("=" * 80)
print("EXAMPLE 1: Simple Inline Change")
print("=" * 80)
print("Scenario: Number changed in a sentence\n")

simple1 = "Employees receive 15 vacation days per year"
simple2 = "Employees receive 20 vacation days per year"

diff = difflib_calc.get_llm_friendly_diff(simple1, simple2, max_context_chars=50)
print(diff)

---

## 2.3.2 Complete LLM-Friendly Diff Examples

This section shows comprehensive examples of using the LLM-friendly diff formats for passing changes to language models.

## 2.4 Token-Based Difflib (Word-Level)

In [12]:
# Token-based comparison
token_difflib = TokenBasedDifflibCalculator()

text1 = "The quick brown fox jumps over the lazy dog"
text2 = "The fast brown fox leaps over the lazy cat"

result = token_difflib.compute_similarity(text1, text2)
print(f"Token-based Difflib: {result.score:.4f}")
print(f"\nMetadata:")
print(f"  Matching tokens: {result.metadata['matching_tokens']}")
print(f"  Text1 tokens: {result.metadata['tokens1_count']}")
print(f"  Text2 tokens: {result.metadata['tokens2_count']}")

Token-based Difflib: 0.6667

Metadata:
  Matching tokens: 6
  Text1 tokens: 9
  Text2 tokens: 9


## 2.5 Factory Functions for Common Use Cases

In [13]:
# FAQ modification detector (preserves numbers and dates)
faq_detector = create_faq_modification_detector()

result = faq_detector.compute_similarity(
    "Deadline is 10 days",
    "Deadline is 12 days"
)
print(f"FAQ modification detector: {result.score:.4f}")

# Character-level analyzer (no preprocessing)
char_analyzer = create_character_level_analyzer()

result = char_analyzer.compute_similarity("ABC123", "ABC456")
print(f"Character-level analyzer: {result.score:.4f}")

# Fast diff calculator (uses quick_ratio)
fast_calc = create_fast_diff_calculator()

result = fast_calc.compute_similarity(faq_original, faq_minor_change)
print(f"Fast diff calculator: {result.score:.4f}")

FAQ modification detector: 0.9474
Character-level analyzer: 0.5000
Fast diff calculator: 0.9952


---

# 3. BM25 Similarity

**BM25** (Best Matching 25) is a probabilistic ranking function:

- **Best for**: Document ranking, term frequency analysis, information retrieval
- **Strengths**: Considers term importance (IDF), handles term frequency saturation
- **Implementation Note**: This version automatically fits corpus from the two texts being compared
- **Limitations**: Simplified compared to full corpus-based BM25

## 3.1 Basic BM25

In [14]:
# Create BM25 calculator (no corpus needed - it auto-fits from the two texts)
bm25_calc = BM25SimilarityCalculator(
    k1=1.5,  # Term frequency saturation parameter
    b=0.75,  # Document length normalization
)

# Compare texts - BM25 automatically fits corpus from both texts
result = bm25_calc.compute_similarity(
    "vacation policy",
    "Employee vacation benefits and time off"
)
print(f"BM25 similarity: {result.score:.4f}")
print(f"\nMetadata:")
pprint(result.metadata)

BM25 similarity: 0.5186

Metadata:
{'avgdl': 4.0,
 'b': 0.75,
 'bm25_raw_score': 0.14883392391343234,
 'doc_length': 6,
 'k1': 1.5,
 'query_length': 2}


## 3.2 BM25 with FAQ Content

**Note**: This implementation automatically fits BM25 from the two texts being compared.
For ranking multiple documents against a query, you would need to call `compute_similarity` 
multiple times (once per document).

In [15]:
# BM25 ranks documents against a query
# Since this implementation auto-fits from two texts, we compare pairwise

faq_corpus = [
    faq_original,
    faq_major_change,  # Different topic
    "How do I request time off for vacation?",
    "What is the company dress code policy?",
]

bm25_faq = BM25SimilarityCalculator()

# Query: vacation-related
query = "How many vacation days do I get?"

print("BM25 Ranking for Query: 'How many vacation days do I get?'")
print("=" * 60)

# Compare query against each document
for i, doc in enumerate(faq_corpus):
    result = bm25_faq.compute_similarity(query, doc)
    doc_preview = doc[:50].replace('\n', ' ')
    print(f"Doc {i}: {result.score:.4f} - {doc_preview}...")

BM25 Ranking for Query: 'How many vacation days do I get?'
Doc 0: 0.6300 -  # Employee Vacation Policy  ## How many vacation ...
Doc 1: 0.5000 -  # Vehicle Parking Policy  ## Where can employees ...
Doc 2: 0.5876 - How do I request time off for vacation?...
Doc 3: 0.5000 - What is the company dress code policy?...


---

# 4. Hybrid Similarity

**Hybrid** combines multiple algorithms with configurable weights:

- **Best for**: Robust similarity detection, FAQ modification detection
- **Strengths**: Combines strengths of multiple algorithms, configurable
- **Features**: Early exit optimization, policy-specific classification

## 4.1 Default Hybrid (Balanced Weights)

In [16]:
# Create hybrid calculator with default weights
hybrid_calc = HybridSimilarityCalculator()

print(f"Default weights: {hybrid_calc.weights}")

result = hybrid_calc.compute_similarity(faq_original, faq_minor_change)
print(f"\nHybrid similarity: {result.score:.4f}")
print(f"\nIndividual algorithm scores:")
pprint(result.metadata['individual_scores'])

Default weights: {'jaccard': 0.25, 'difflib': 0.25, 'bm25': 0.5}

Hybrid similarity: 0.9871

Individual algorithm scores:
{'bm25': 0.9997777100061634,
 'difflib': 0.995475113122172,
 'jaccard': 0.9534883720930233}


## 4.2 Custom Weighted Hybrid

In [17]:
# Create custom weighted hybrid
custom_hybrid = HybridSimilarityCalculator(
    weights={
        "jaccard": 0.3,
        "difflib": 0.5,  # Emphasize difflib for character-level changes
        "bm25": 0.2,
    },
    early_exit_threshold=0.3,  # Enable early exit for very different texts
)

result = custom_hybrid.compute_similarity(faq_original, faq_major_change)
print(f"Custom hybrid similarity: {result.score:.4f}")
print(f"Early exit triggered: {result.metadata['early_exit']}")
print(f"\nThis correctly identified completely different content!")

Custom hybrid similarity: 0.0000
Early exit triggered: True

This correctly identified completely different content!


## 4.3 Factory Method: Modification Detection

In [18]:
# Optimized for detecting FAQ modifications
mod_detector = HybridSimilarityCalculator.for_modification_detection()

print(f"Modification detection weights: {mod_detector.weights}")
print(f"Early exit threshold: {mod_detector.early_exit_threshold}")

# Test on different scenarios
scenarios = [
    ("Identical", faq_original, faq_identical),
    ("Minor change", faq_original, faq_minor_change),
    ("Major change", faq_original, faq_major_change),
]

print("\nModification Detection Results:")
print("=" * 60)
for label, text1, text2 in scenarios:
    result = mod_detector.compute_similarity(text1, text2)
    print(f"{label:15} → Score: {result.score:.4f} | Early exit: {result.metadata['early_exit']}")

Modification detection weights: {'jaccard': 0.2, 'difflib': 0.5, 'bm25': 0.3}
Early exit threshold: 0.3

Modification Detection Results:
Identical       → Score: 0.9999 | Early exit: False
Minor change    → Score: 0.9884 | Early exit: False
Major change    → Score: 0.0000 | Early exit: True


## 4.4 Factory Method: General Similarity

In [19]:
# Balanced weights for general similarity
general_sim = HybridSimilarityCalculator.for_general_similarity()

print(f"General similarity weights: {general_sim.weights}")

result = general_sim.compute_similarity(faq_original, faq_minor_change)
print(f"\nGeneral similarity: {result.score:.4f}")

General similarity weights: {'jaccard': 0.33, 'difflib': 0.33, 'bm25': 0.34}

General similarity: 0.9831


## 4.5 Algorithm Breakdown

In [20]:
# Get detailed breakdown of individual algorithm scores
breakdown = hybrid_calc.get_algorithm_breakdown(faq_original, faq_minor_change)

print("Algorithm Breakdown:")
print("=" * 60)
for algo, score in breakdown.items():
    print(f"{algo:10} → {score:.4f}")

Algorithm Breakdown:
jaccard    → 0.9535
difflib    → 0.9955
bm25       → 0.9998


---

# 5. Policy Content Similarity

**PolicyContentSimilarity** is specialized for policy/FAQ classification:

- Detects modifications vs. new content
- Classifies change severity
- Optimized for policy documents

## 5.1 Modification Detection

In [21]:
# Create policy similarity calculator
policy_sim = PolicyContentSimilarity()

# Check if content is a modification
is_mod = policy_sim.is_modification(faq_original, faq_minor_change)
print(f"Is modification (15→20 days): {is_mod}")

is_mod = policy_sim.is_modification(faq_original, faq_major_change)
print(f"Is modification (vacation→parking): {is_mod}")

# Custom threshold
is_mod = policy_sim.is_modification(
    faq_original,
    faq_minor_change,
    threshold=0.95  # Very strict
)
print(f"\nIs modification (strict threshold): {is_mod}")

Is modification (15→20 days): True
Is modification (vacation→parking): False

Is modification (strict threshold): True


## 5.2 Change Classification

In [22]:
# Classify the type and severity of change
change_type, score, details = policy_sim.classify_change(faq_original, faq_minor_change)

print(f"Change Classification:")
print("=" * 60)
print(f"Type: {change_type}")
print(f"Score: {score:.4f}")
print(f"\nDetails:")
print(f"  Similarity: {details['similarity_score']:.4f}")
print(f"  Early exit: {details['early_exit']}")
print(f"\nAlgorithm breakdown:")
pprint(details['breakdown'])

Change Classification:
Type: identical
Score: 0.9884

Details:
  Similarity: 0.9884
  Early exit: False

Algorithm breakdown:
{'bm25': 0.9997777100061634,
 'difflib': 0.995475113122172,
 'jaccard': 0.9534883720930233}


In [23]:
# Test different scenarios
test_cases = [
    ("Identical", faq_original, faq_identical),
    ("Minor modification", faq_original, faq_minor_change),
    ("Major change", faq_original, faq_major_change),
]

print("Change Classification Results:")
print("=" * 60)
for label, text1, text2 in test_cases:
    change_type, score, _ = policy_sim.classify_change(text1, text2)
    print(f"{label:20} → {change_type:20} (score: {score:.4f})")

Change Classification Results:
Identical            → identical            (score: 0.9999)
Minor modification   → identical            (score: 0.9884)
Major change         → new_content          (score: 0.0000)


---

# 6. Real-World FAQ Examples

Practical examples using different scenarios

## 6.1 Detecting Numeric Changes

In [24]:
# Different algorithms on numeric changes
text1 = "Employees receive 10 sick days and 15 vacation days per year"
text2 = "Employees receive 12 sick days and 15 vacation days per year"

algorithms = [
    ("Jaccard", JaccardSimilarityCalculator()),
    ("Difflib", DifflibSimilarityCalculator(remove_punctuation=False)),
    ("BM25", BM25SimilarityCalculator()),  # No corpus parameter needed
    ("Hybrid", HybridSimilarityCalculator.for_modification_detection()),
]

print("Numeric Change Detection (10→12 sick days):")
print("=" * 60)
for name, calc in algorithms:
    result = calc.compute_similarity(text1, text2)
    print(f"{name:15} → {result.score:.4f}")

Numeric Change Detection (10→12 sick days):
Jaccard         → 0.8182
Difflib         → 0.9833
BM25            → 0.7290
Hybrid          → 0.8740


## 6.2 Detecting Date Changes

In [25]:
text1 = "The deadline for submission is January 15, 2024 at 5:00 PM EST"
text2 = "The deadline for submission is February 15, 2024 at 5:00 PM EST"

print("Date Change Detection (January→February):")
print("=" * 60)
for name, calc in algorithms:
    result = calc.compute_similarity(text1, text2)
    print(f"{name:15} → {result.score:.4f}")

Date Change Detection (January→February):
Jaccard         → 0.8462
Difflib         → 0.9440
BM25            → 0.7316
Hybrid          → 0.8607


## 6.3 Detecting Phrase Changes

In [26]:
text1 = "Employees must submit their timesheets by Friday"
text2 = "Employees are required to submit their timesheets by Friday"

print("Phrase Change Detection (must→are required to):")
print("=" * 60)
for name, calc in algorithms:
    result = calc.compute_similarity(text1, text2)
    print(f"{name:15} → {result.score:.4f}")

Phrase Change Detection (must→are required to):
Jaccard         → 0.6000
Difflib         → 0.8598
BM25            → 0.6266
Hybrid          → 0.7379


## 6.4 Batch Comparison

In [27]:
# Compare one FAQ against multiple versions
original = "Employees receive 15 vacation days per year"

variants = [
    ("Identical", "Employees receive 15 vacation days per year"),
    ("Number change", "Employees receive 20 vacation days per year"),
    ("Word change", "Staff members receive 15 vacation days per year"),
    ("Phrase change", "Employees get 15 vacation days annually"),
    ("Major rewrite", "Full-time workers are entitled to three weeks paid leave"),
    ("Different topic", "Parking is available in the underground garage"),
]

hybrid = HybridSimilarityCalculator.for_modification_detection()

print("Batch Comparison Results:")
print("=" * 70)
print(f"{'Variant':<20} {'Score':<10} {'Classification'}")
print("=" * 70)

for label, text in variants:
    result = hybrid.compute_similarity(original, text)
    change_type, _, _ = policy_sim.classify_change(original, text)
    print(f"{label:<20} {result.score:<10.4f} {change_type}")

Batch Comparison Results:
Variant              Score      Classification
Identical            0.8963     minor_modification
Number change        0.8168     major_modification
Word change          0.7445     major_modification
Phrase change        0.6327     significant_change
Major rewrite        0.0000     new_content
Different topic      0.0000     new_content


---

# 7. Best Practices & Recommendations

## When to Use Each Algorithm

### Jaccard
- ✅ Quick keyword matching
- ✅ Finding similar FAQs based on terminology
- ✅ Large-scale FAQ deduplication
- ❌ Not good for: detecting small character changes, word order importance

### Difflib
- ✅ Detecting precise character-level changes ("10" → "12")
- ✅ Generating detailed diffs for review
- ✅ Typo detection and correction
- ❌ Not good for: semantic similarity, large text comparisons (slower)

### BM25
- ✅ Ranking FAQs by relevance to query
- ✅ Finding semantically related content
- ✅ Term importance weighting
- ❌ Not good for: exact change detection, requires corpus initialization

### Hybrid
- ✅ **Recommended for FAQ modification detection**
- ✅ Robust similarity across different types of changes
- ✅ Configurable for specific use cases
- ✅ Early exit optimization for performance

### PolicyContentSimilarity
- ✅ **Best for policy/FAQ content classification**
- ✅ Automatic change type detection
- ✅ Pre-configured for policy documents

## Recommended Configuration

For FAQ modification detection:
```python
detector = HybridSimilarityCalculator.for_modification_detection()
# or
policy_sim = PolicyContentSimilarity()
```

For general FAQ similarity:
```python
similarity = HybridSimilarityCalculator.for_general_similarity()
```

For fast keyword matching:
```python
jaccard = JaccardSimilarityCalculator.for_faq_content()
```

---

# Summary

This notebook demonstrated:

1. ✅ **Jaccard Similarity** - Token set overlap, fast keyword matching
2. ✅ **Difflib Similarity** - Character-level precision, detailed diffs
3. ✅ **BM25 Similarity** - Term importance, document ranking
4. ✅ **Hybrid Similarity** - Combined algorithms, robust detection
5. ✅ **Policy Classification** - Specialized FAQ/policy content analysis

**Key Takeaway**: Use `HybridSimilarityCalculator.for_modification_detection()` or `PolicyContentSimilarity()` for most FAQ modification detection tasks!